<h1>Capstone Project Week 3 Assignment</h1>
<p>By Andres Lopez</p>
<br>
<p>This notebook has three parts:</p>
<ul><li>Web Scraping and Dataframe preprocessing</li><li>Adding Latitude and Longitude to the Dataframe</li>
    <li>Exploring and Clustering</li></ul>

<h3>Part I - Web scraping and dataframe pre-processing</h3>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all('table', attrs={'class':'wikitable sortable'})

<p><strong>The cells of the first row will be store in a list to create the headers of the dataframe</strong></p>

In [3]:
table_obj = results[0] #gets the first row of the table
headers = table_obj.find_all('th') #gets each heaader
col_head = [] 
for x in headers:
    header_text = x
    col_head.append(header_text.text[0:-1])

<p><strong>The cells of all the other rows will be store in three lists, each with the content of one column</strong></p>

In [4]:
rows = table_obj.find_all('tr')
rows.pop(0) #the first row is the header row, already stored in the col_head list so it must be deleted here
postal = []
borough = []
neigh = []
for x in rows: #iteration to get the text in each row
    c_content =[]
    cells_obj = x
    cells_list = cells_obj.find_all('td')
    for y in cells_list: #iteration to get the text of each cell in a row
        cell_text = y
        c_content.append(cell_text.text[0:-1])
    if 'Not assigned' not in c_content:  #if the Borough column says 'Not Assigned' the row will be ignored
        postal.append(c_content[0])
        borough.append(c_content[1])
        if c_content[2] == "":       #If the Neighnorhood cell is blank, the text of the Bourough column will be copied
            c_content[2] = c_content[1]
        neigh.append(c_content[2])

<p><strong>Making sure that no postal code is listed twice</strong></p>
<p>The assignment instructions indicate that there is at least one postal code listed twice. However, that doesn't seem the case. I'm assuming the table has been edited since the assignment isntructions were posted.</p>

In [5]:
if len(postal) == len(set(postal)):
    print("There are no duplicate zip codes")
else:
    print("There are uplicate zip codes")

There are no duplicate zip codes


<p><strong>Create a dataframe using the lists with the data in the Wikipidia table</strong></p>

In [6]:
table = {col_head[0]:postal, col_head[1]:borough, col_head[2]:neigh}
df_toronto = pd.DataFrame(table)
df_toronto.sort_values(by=['Postal Code'], inplace=True) #sorting is ncessary to add the coordenates columns later 
df_toronto.reset_index(inplace=True)
df_toronto.head()

,index,Postal Code,Borough,Neighborhood
0,6,M1B,Scarborough,Malvern / Rouge
1,12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,18,M1E,Scarborough,Guildwood / Morningside / West Hill
3,22,M1G,Scarborough,Woburn
4,26,M1H,Scarborough,Cedarbrae


In [7]:
df_toronto.shape

(103, 4)

<h3>Part II - Adding Latitude and Longitude</h3>
<p>Since the assignment gives the option, I used the csv file instead of the Geocoder Python package</p>

In [8]:
location = "http://cocl.us/Geospatial_data"
df_coord = pd.read_csv(location)
#sort the dataframe by postal code to make sure it aligns correctly with the df_toronto dataframe
df_coord.sort_values(by=['Postal Code'], inplace=True) 
#let's make sure it has the same number of rows than df_toronto
df_coord.shape

(103, 3)

In [9]:
#let's take a look at the dataframe
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<p><strong>Now we add the two columns with the coordentaes to the intitial dataframe</strong></p>
<p>The dataframe with the Neighborhoods (df_toronto) and the dataframe with the coordinates (df_coord)
both have the same number of rows and both are sorted by zip code, so it should be safe to add the coordinates from  
df_coord to df_toronto</p>

In [10]:
df_toronto['Latitude'] = df_coord['Latitude']
df_toronto['Longitude'] = df_coord['Longitude']
df_toronto.head()

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,18,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,22,M1G,Scarborough,Woburn,43.770992,-79.216917
4,26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h3>Part III - Exploring and Clustering</h3>
<p><strong>A. Exploring</strong></p>

In [11]:
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

import folium
from IPython.core.display import HTML
m = folium.Map(location=[45.5236, -122.6750])
HTML(m._repr_html_())

import json
from pandas.io.json import json_normalize
%matplotlib inline

<p><strong>Map with all the neighborhoods in Toronto</strong></p>

In [12]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="tnto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto Coordinates: {}, {}.'.format(latitude, longitude))

Toronto Coordinates: 43.6534817, -79.3839347.


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
        df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        parse_html=False).add_to(map_toronto)  
map_toronto

<p><strong>Now let's focus on Downtown Toronto and explore it</strong></p>

In [14]:
downtown = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print('Downtown Toronto has {} neighborhoods.'.format(downtown.shape[0]))

Downtown Toronto has 19 neighborhoods.


In [15]:
address = 'Downtown Toronto, ON'
geolocator = Nominatim(user_agent="dtwn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Downtown Toronto Coordinates: {}, {}.'.format(latitude, longitude))

Downtown Toronto Coordinates: 43.6563221, -79.3809161.


In [16]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, borough, neighborhood in zip(downtown['Latitude'], downtown['Longitude'], downtown['Borough'], downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#f5425a',
        fill=True,
        fill_color='#f5425a',
        fill_opacity=1,
        parse_html=False).add_to(map_downtown)  
map_downtown

<p><strong>Getting all the venues in Downtown Toronto</strong></p>

In [17]:
CLIENT_ID = '2IGBWCWEV040N5NA1OFGXZHSE3IGB0JBLMQK54LGUG2SYUMM'
CLIENT_SECRET = 'E23X3LXXJKJOH5XIJI0VQBO2CCSEQVXNIIBYUGGFSMZSOGCS' 
VERSION = '20200430'
LIMIT = 100

In [18]:
#These functions will be use to get the venues info from Fousquare, clean it, and ut it in a dataframe
def category(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                  'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(venues)

In [19]:
downtown_venues = getVenues(names=downtown['Neighborhood'], 
                            latitudes=downtown['Latitude'], longitudes=downtown['Longitude'])
print('Downtown Toronto has {} venues.'.format(downtown_venues.shape[0]))
downtown_venues.head()

Downtown Toronto has 1223 venues.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,St. James Town / Cabbagetown,43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


<p><strong>B. Clustering</strong></p>
<p>Before clustering:</p>
<ul><li>Convert categorical Venue Category to numeric, and put the categories as columns</li>
    <li>Add the Neighborhoods column, and lace it as the first column</li>
    <li>Group categories by neighborhood and get the mean</li></ul>

In [20]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()

downtown_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.017544,0.000000,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0
1,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Central Bay Street,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.030303,0.000000,0.000000,0.0,0.0,0.015152,0.0,0.0,0.0,0.0
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Church and Wellesley,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.012821,0.012821,0.012821,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


<p><strong>K-Means Clustering</strong></p>

In [21]:
kclusters = 4
downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

#cluster labels
kmeans.labels_ 

array([1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])